In [17]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('hw3_course_lead_scoring.csv')

In [3]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [7]:
df.head

<bound method NDFrame.head of          lead_source       industry  number_of_courses_viewed  annual_income  \
0           paid_ads            NaN                         1        79450.0   
1       social_media         retail                         1        46992.0   
2             events     healthcare                         5        78796.0   
3           paid_ads         retail                         2        83843.0   
4           referral      education                         3        85012.0   
...              ...            ...                       ...            ...   
1457        referral  manufacturing                         1            NaN   
1458        referral     technology                         3        65259.0   
1459        paid_ads     technology                         1        45688.0   
1460        referral            NaN                         5        71016.0   
1461  organic_search        finance                         3        92855.0   

     empl

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [11]:
fillvs = {"lead_source":"NA", "industry":"NA", "number_of_courses_viewed":0.0, "annual_income":0.0, 
          "employment_status":"NA", "location":"NA", "interaction_count":0.0, "lead_score":0.0, 
          "converted":0.0}
df = df.fillna(value=fillvs)

In [12]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [13]:
df.head

<bound method NDFrame.head of          lead_source       industry  number_of_courses_viewed  annual_income  \
0           paid_ads             NA                         1        79450.0   
1       social_media         retail                         1        46992.0   
2             events     healthcare                         5        78796.0   
3           paid_ads         retail                         2        83843.0   
4           referral      education                         3        85012.0   
...              ...            ...                       ...            ...   
1457        referral  manufacturing                         1            0.0   
1458        referral     technology                         3        65259.0   
1459        paid_ads     technology                         1        45688.0   
1460        referral             NA                         5        71016.0   
1461  organic_search        finance                         3        92855.0   

     empl

In [14]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [16]:
df.corr(numeric_only=True)

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [46]:
df_train_full, df_test = sk.model_selection.train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = sk.model_selection.train_test_split(df_train_full, test_size=0.25, random_state=42)

In [47]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [48]:
df_train_full = df_train_full.drop(columns="converted")
df_train = df_train.drop(columns="converted")
df_val = df_val.drop(columns="converted")
df_test = df_test.drop(columns="converted")

In [49]:
df_train.iloc[:,[0,1,4,5]].aggregate(func=(lambda a: sk.metrics.mutual_info_score(a,y_train)), axis=0)

lead_source          0.035396
industry             0.011575
employment_status    0.012938
location             0.004464
dtype: float64

In [99]:
dv = sk.feature_extraction.DictVectorizer(sparse=False)
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
model = sk.linear_model.LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [100]:
model.score(X_train,y_train)

0.7385844748858448

In [102]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'annual_income': np.float64(-0.0),
 'employment_status=NA': np.float64(-0.015),
 'employment_status=employed': np.float64(0.034),
 'employment_status=self_employed': np.float64(0.003),
 'employment_status=student': np.float64(0.012),
 'employment_status=unemployed': np.float64(-0.103),
 'industry=NA': np.float64(-0.025),
 'industry=education': np.float64(0.049),
 'industry=finance': np.float64(-0.02),
 'industry=healthcare': np.float64(-0.013),
 'industry=manufacturing': np.float64(-0.003),
 'industry=other': np.float64(-0.009),
 'industry=retail': np.float64(-0.032),
 'industry=technology': np.float64(-0.016),
 'interaction_count': np.float64(0.311),
 'lead_score': np.float64(0.051),
 'lead_source=NA': np.float64(0.02),
 'lead_source=events': np.float64(-0.012),
 'lead_source=organic_search': np.float64(-0.012),
 'lead_source=paid_ads': np.float64(-0.115),
 'lead_source=referral': np.float64(0.08),
 'lead_source=social_media': np.float64(-0.03),
 'location=NA': np.float64(0.004),
 'l

In [94]:
np.mean(model.predict(X_train)==y_train)

np.float64(0.7385844748858448)

In [96]:
X_val = dv.transform(df_val.to_dict(orient='records'))
val_score = model.score(X_val,y_val)

In [91]:
df_train.iloc[:,[0,1,4,5]].aggregate(func=(lambda a: a.name), axis=0)

lead_source                lead_source
industry                      industry
employment_status    employment_status
location                      location
dtype: object

In [97]:

def fit_ko(col):
    df_train_copy = df_train.copy().drop(columns=col.name)
    df_val_copy = df_val.copy().drop(columns=col.name)
    X_train = dv.fit_transform(df_train_copy.to_dict(orient='records'))
    X_val = dv.transform(df_val_copy.to_dict(orient='records'))
    model.fit(X_train, y_train)
    return abs(model.score(X_val,y_val) - val_score)

In [98]:
df_train.aggregate(func=fit_ko, axis=0)

lead_source                 0.003413
industry                    0.000000
number_of_courses_viewed    0.143345
annual_income               0.153584
employment_status           0.003413
location                    0.010239
interaction_count           0.143345
lead_score                  0.006826
dtype: float64

In [108]:
score_diffs = np.zeros(df_train.shape[1])
for idx in range(df_train.shape[1]):
    col = df_train.columns[idx]
    df_train_copy = df_train.copy().drop(columns=col)
    df_val_copy = df_val.copy().drop(columns=col)
    dv = sk.feature_extraction.DictVectorizer(sparse=False)
    X_train = dv.fit_transform(df_train_copy.to_dict(orient='records'))
    X_val = dv.transform(df_val_copy.to_dict(orient='records'))
    model.fit(X_train, y_train)
    score_diffs[idx] = abs(model.score(X_val,y_val) - val_score)

In [109]:
score_diffs

array([0.00341297, 0.        , 0.14334471, 0.15358362, 0.00341297,
       0.01023891, 0.14334471, 0.00682594])

In [118]:
dv = sk.feature_extraction.DictVectorizer(sparse=False)
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
X_val = dv.transform(df_val.to_dict(orient='records'))
Cs = [0.01, 0.1, 1, 10, 100]
val_scores = np.zeros_like(Cs)
for idx in range(val_scores.shape[0]):
    model = sk.linear_model.LogisticRegression(solver='liblinear', C=Cs[idx], max_iter=1000, random_state=42)
    
    model.fit(X_train, y_train)
    val_scores[idx] = model.score(X_val,y_val)

val_scores
    

array([0.6996587, 0.6996587, 0.6996587, 0.6996587, 0.6996587])